In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import optuna
from joblib import parallel_backend
from sklearn.preprocessing import StandardScaler

In [15]:
df = pd.read_csv('df_train.csv')

In [16]:
to_drop_col= ['Unnamed: 0','proppant_to_frac_fluid_ratio', 'bh_x', 'bh_y', 'horizontal_midpoint_x', 'well_family_relationship', 'proppant_to_frac_fluid_ratio']
df_final = df.drop(columns=to_drop_col)

In [17]:



features = df_final.drop(columns=['OilPeakRate'])
target =  df_final['OilPeakRate']


# add standard scalar
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features = scaler.fit_transform(features.values)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=42)
# Create the Random Forest model
rf_model = RandomForestRegressor(random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict using tahe model
rf_predictions = rf_model.predict(X_test)

# Evaluate the model
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_rmse = np.sqrt(rf_mse)
rf_r2_score = r2_score(y_test, rf_predictions)

print("Random Forest RMSE:", rf_rmse)
print("Random Forest R2 Score:", rf_r2_score)


Random Forest RMSE: 92.27442416877946
Random Forest R2 Score: 0.6602695635424444


In [18]:
# Assuming df_final is your DataFrame and is already preprocessed
features = df_final.drop(columns=['OilPeakRate'])
target = df_final['OilPeakRate']

# Standardize features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)


def objective(trial):
    # Select features: create a boolean mask for selected features
    feature_mask = []
    for i in range(features.shape[1]):
        feature_mask.append(trial.suggest_categorical(f"feature_{i}", [True, False]))
    
    # Subset the features
    X_subset = features_scaled[:, feature_mask]

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X_subset, target, test_size=0.1, random_state=42)

    # Create and train the model
    rf_model = RandomForestRegressor(random_state=42, n_jobs=16)
    rf_model.fit(X_train, y_train)

    # Predict and calculate the MSE
    predictions = rf_model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    return rmse


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=500, n_jobs=16)  # Use all available CPU cores for parallel trial execution


[I 2024-01-20 20:00:50,934] A new study created in memory with name: no-name-0534376c-1929-481c-8972-e3fe0addae5f
[I 2024-01-20 20:00:58,735] Trial 4 finished with value: 97.99203155787639 and parameters: {'feature_0': True, 'feature_1': False, 'feature_2': True, 'feature_3': False, 'feature_4': True, 'feature_5': False, 'feature_6': True, 'feature_7': True, 'feature_8': False, 'feature_9': False, 'feature_10': False, 'feature_11': True, 'feature_12': True, 'feature_13': True, 'feature_14': True, 'feature_15': False, 'feature_16': False, 'feature_17': True, 'feature_18': False, 'feature_19': False, 'feature_20': False, 'feature_21': False}. Best is trial 4 with value: 97.99203155787639.
[I 2024-01-20 20:00:58,904] Trial 8 finished with value: 99.02514505695869 and parameters: {'feature_0': True, 'feature_1': True, 'feature_2': False, 'feature_3': False, 'feature_4': True, 'feature_5': False, 'feature_6': False, 'feature_7': False, 'feature_8': True, 'feature_9': False, 'feature_10': Tr

In [6]:
# # Get the best parameters
# best_features = [f"{features.columns[i]}" for i in range(features.shape[1]) if study.best_params[f"feature_{i}"]]

# # Print out the selected features
# print("Selected Features:", best_features)

In [7]:



# features = df_final[best_features]
# target =  df_final['OilPeakRate']


# # add standard scalar
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# features = scaler.fit_transform(features.values)

# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=42)
# # Create the Random Forest model
# rf_model = RandomForestRegressor(random_state=42)

# # Train the model
# rf_model.fit(X_train, y_train)

# # Predict using tahe model
# rf_predictions = rf_model.predict(X_test)

# # Evaluate the model
# rf_mse = mean_squared_error(y_test, rf_predictions)
# rf_rmse = np.sqrt(rf_mse)
# rf_r2_score = r2_score(y_test, rf_predictions)

# print("Random Forest RMSE:", rf_rmse)
# print("Random Forest R2 Score:", rf_r2_score)


In [8]:


# Load and prepare your DataFrame
# df_final = pd.read_csv('path_to_your_data.csv')  # Replace with your file path

def objective(trial):
    
    n_estimators = trial.suggest_int('n_estimators', 100,300, 1000)
    max_depth = trial.suggest_int('max_depth', 10, 100)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 7, 14)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, ,4,8)
    max_features = trial.suggest_categorical('max_features', [ 'sqrt'])

    # Create and train the model with all available cores
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth,
                                  min_samples_split=min_samples_split,
                                  min_samples_leaf=min_samples_leaf,
                                  max_features=1.0, random_state=42, n_jobs=12)

    with parallel_backend('threading'):
        
        scores = cross_val_score(model, X_train, y_train, cv=5,n_jobs=12)

    return scores.mean()
    # return avg_rmse


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500,n_jobs=12) 

# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters:", best_params)

# # Train the final model using the best hyperparameters
# final_model = RandomForestRegressor(**best_params, random_state=42, n_jobs=-1)
# final_model.fit(X_train, y_train)

# # Predict and evaluate the final model
# final_predictions = final_model.predict(X_test)
# final_mse = mean_squared_error(y_test, final_predictions)
# final_rmse = np.sqrt(final_mse)
# final_r2_score = r2_score(y_test, final_predictions)

# print("Final Model - Random Forest RMSE:", final_rmse)
# print("Final Model - Random Forest R2 Score:", final_r2_score)


[I 2024-01-20 18:37:41,834] A new study created in memory with name: no-name-97939567-ff7b-42ba-9301-415e5987c326
[I 2024-01-20 18:38:55,533] Trial 5 finished with value: 0.5709132135768703 and parameters: {'n_estimators': 154, 'max_depth': 70, 'min_samples_split': 14, 'min_samples_leaf': 4, 'max_features': 'sqrt'}. Best is trial 5 with value: 0.5709132135768703.
[I 2024-01-20 18:39:01,754] Trial 6 finished with value: 0.56952247006812 and parameters: {'n_estimators': 168, 'max_depth': 17, 'min_samples_split': 12, 'min_samples_leaf': 6, 'max_features': 'sqrt'}. Best is trial 5 with value: 0.5709132135768703.
[I 2024-01-20 18:39:19,765] Trial 2 finished with value: 0.5711420231626142 and parameters: {'n_estimators': 221, 'max_depth': 33, 'min_samples_split': 4, 'min_samples_leaf': 6, 'max_features': 'sqrt'}. Best is trial 2 with value: 0.5711420231626142.
[I 2024-01-20 18:39:34,346] Trial 11 finished with value: 0.5656155606016015 and parameters: {'n_estimators': 274, 'max_depth': 42, '

Best hyperparameters: {'n_estimators': 832, 'max_depth': 57, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'sqrt'}


In [10]:
# Train the final model using the best hyperparameters
final_model = RandomForestRegressor(**best_params, random_state=42, n_jobs=-1)
final_model.fit(X_train, y_train)

# Predict and evaluate the final model
final_predictions = final_model.predict(X_test)
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_r2_score = r2_score(y_test, final_predictions)

print("Final Model - Random Forest RMSE:", final_rmse)
print("Final Model - Random Forest R2 Score:", final_r2_score)

Final Model - Random Forest RMSE: 93.21442475398418
Final Model - Random Forest R2 Score: 0.6533126324811858


In [12]:
best_params

{'n_estimators': 832,
 'max_depth': 57,
 'min_samples_split': 4,
 'min_samples_leaf': 2,
 'max_features': 'sqrt'}

In [13]:
target.max()

2448.143236

In [20]:
features = df_final.drop(columns=['OilPeakRate'])
target = df_final['OilPeakRate']

# Standard scaling
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features.values)

# Random Forest model
rf_model = RandomForestRegressor(**best_params, random_state=42, n_jobs=-1)

# Cross-validation (using R2 as the metric)
cv_scores = cross_val_score(rf_model, features_scaled, target, cv=10, scoring='neg_root_mean_squared_error', n_jobs=12)

# Calculate average R2 score across all folds
average_r2_score = np.mean(cv_scores)